# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298619444782                   -0.85    5.2
  2   -8.300227850697       -2.79       -1.25    1.0
  3   -8.300447106130       -3.66       -1.89    2.5
  4   -8.300462441711       -4.81       -2.79    2.4
  5   -8.300464534181       -5.68       -3.17    3.4
  6   -8.300464591442       -7.24       -3.30    1.8
  7   -8.300464616840       -7.60       -3.44    1.2
  8   -8.300464630630       -7.86       -3.57    1.0
  9   -8.300464639802       -8.04       -3.75    1.0
 10   -8.300464641735       -8.71       -3.85    1.5
 11   -8.300464643505       -8.75       -4.09    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64160539189                   -0.70    6.0
  2   -16.67578399459       -1.47       -1.14    1.4
  3   -16.67921741444       -2.46       -1.88    3.4
  4   -16.67927661896       -4.23       -2.71    4.5
  5   -16.67928566588       -5.04       -3.07    5.6
  6   -16.67928621511       -6.26       -3.50    1.2
  7   -16.67928622170       -8.18       -4.03    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32527199502                   -0.56    7.8
  2   -33.33452733715       -2.03       -1.00    2.0
  3   -33.33598357161       -2.84       -1.72    6.1
  4   -33.33616401381       -3.74       -2.62    4.1
  5   -33.33690022854       -3.13       -2.50    8.6
  6   -33.33692575764       -4.59       -2.70    3.0
  7   -33.33694386013       -4.74       -3.59    3.1
  8   -33.33694390828       -7.32       -3.82    4.1
  9   -33.33694392525       -7.77       -4.52    1.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298529473651                   -0.85    5.2
  2   -8.300270288078       -2.76       -1.59    1.0
  3   -8.300390158389       -3.92       -2.29    5.0
  4   -8.300307177166   +   -4.08       -2.16    6.2
  5   -8.300464473841       -3.80       -3.71    2.1
  6   -8.300464603472       -6.89       -3.95    3.5
  7   -8.300464633428       -7.52       -4.27    1.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.29681437766                   -0.56    6.8
  2   -33.30759107280       -1.97       -1.25    2.5
  3   -15.54609848048   +    1.25       -0.44    7.8
  4   -33.18378896834        1.25       -1.32    6.4
  5   -33.14336334598   +   -1.39       -1.27    4.0
  6   -25.34665382782   +    0.89       -0.62    5.2
  7   -33.33625158981        0.90       -2.21    5.5
  8   -33.33608147149   +   -3.77       -2.45    4.1
  9   -33.33646753871       -3.41       -2.55    2.6
 10   -33.33641835164   +   -4.31       -2.57    2.1
 11   -33.33667274634       -3.59       -2.61    2.6
 12   -33.33693519669       -3.58       -3.34    1.9
 13   -33.33694025055       -5.30       -3.58    4.0
 14   -33.33694120040       -6.02       -3.70    1.5
 15   -33.33694242489       -5.91       -3.89    3.2
 16   -33.33694345984       -5.99       -4.16    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.